In [21]:
API_URLS = {
    "prod": "https://api.satws.com",
    "test": "https://api.sandbox.satws.com",
}
API_KEYS = {
    "prod": "3a5faf1c2155c2ded9ca82aafa1bfa49",
    "test": "",
}

ACCEPT_FORMAT = {
    "pdf": "application/pdf",
    "json": "application/json",
    "xml": "text/xml",
}

def get_metadata() -> dict[str, str]:
    with open("credentials/metadata.json") as cfile:
        credentials = loads(cfile.read())
    return credentials


def _is_valid_date(date_string: str) -> bool:
    if not isinstance(date_string, str):
        return False

    try:
        dt.strptime(date_string, '%Y-%m-%d')
        return True
    except ValueError:
        pass
    try:
        dt.strptime(date_string, '%Y-%m-%dT%H:%M:%SZ')
        return True
    except ValueError:
        pass
    try:
        dt.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%fZ')
        return True
    except ValueError:
        pass

    return False
    
def _parse_arg(value: Union[str, tuple, int, float]) -> str:

    logic_map_date = {
        "<=": "[before]",
        "<": "[strictly_before]",
        ">=": "[after]",
        ">": "[strictly_after]",
    }
    logic_map_number = {
        "<=": "[lte]",
        "<": "[lt]",
        ">=": "[gte]",
        ">": "[gt]",
        "between": "[between]",
    }
    if isinstance(value, (tuple, list)):
        if _is_valid_date(value[1]):
            suffix = logic_map_date[value[0]]
            parsed_value = value[1]
        else:
            suffix = logic_map_number[value[0]]
            parsed_value = f"{value[1]}..{value[2]}" if value[0] == "between" else value[1]
    else:
        suffix = ""
        parsed_value = value
    
    return f"{suffix}={parsed_value}"

def _parse_args(**args) -> str:
    if not args:
        return ""
    return "?" + "&".join(f"{key}{_parse_arg(value)}" for key, value in args.items())

def get_all_credentials(env: str = "prod") -> DataFrame:
    http_request = {
        "url": f"{API_URLS[env]}/credentials",
        "headers": {
            "X-API-Key": API_KEYS[env],
        }
    }
    http_response = get(**http_request)
    parsed_content = loads(http_response.text)
    return DataFrame(parsed_content["hydra:member"])


def get_credential(_id="credentials", env: str = "prod") -> DataFrame:
    http_request = {
        "url": f"{API_URLS[env]}/credentialss",
        "headers": {
            "X-API-Key": API_KEYS[env],
        }
    }
    response = get(**http_request)
    parsed_response = loads(response.text)
    return DataFrame(parsed_response["hydra:member"])


def get_invoices(rfc: str, env: str = "prod", **query_args) -> Union[DataFrame, None]:
    parsed_query_args = _parse_args(**query_args)
    http_request = {
        "url": f"{API_URLS[env]}/taxpayers/{rfc}/invoices{parsed_query_args}",
        "headers": {
            "X-API-Key": API_KEYS[env],
        }
    }
    response = get(**http_request)

    if (response.status_code - 200) >= 100:
        raise ConnectionError
        
    parsed_response = response.json()
    
    if not parsed_response:
        return None
    elif parsed_response['hydra:totalItems'] == 0:
        return None
    
    return DataFrame(parsed_response["hydra:member"]).drop('@id', axis=1)


def retrieve_cfdi(invoice_id: str, format: str = "json", env: str = "prod") -> None:
    http_request = {
        "url": f"{API_URLS[env]}/invoices/{invoice_id}/cfdi",
        "headers": {
            "X-API-Key": API_KEYS[env],
            "Accept": ACCEPT_FORMAT[format],
        },
    }
    response = get(**http_request)

    with open(f"{invoice_id}.{format}", "wb") as file:
        file.write(response.content)


In [7]:
retrieve_cfdi("985213d7-ee6d-4ff7-9786-66a046b8adac", "json")


In [25]:
df_invoices = get_invoices(
    rfc="RIGC6106151I7",
    type="I", 
    version=3.3,
    paymentMethod="03",
    issuedAt=(">", "2023-01-19T23:37:41Z"),
    #tax=("between", 10, 12.1)
    )


In [26]:
df_invoices

,@type,id,uuid,version,type,usage,paymentType,paymentMethod,placeOfIssue,issuer,...,reference,relations,creditedAmount,appliedCreditNotes,issuedCreditNotes,createdAt,updatedAt,items,xml,pdf
0,Invoice,985213d7-ee6d-4ff7-9786-66a046b8adac,1891b321-c2e6-4974-a80a-c9dd5d7273b9,3.3,I,P01,PUE,03,06600,"{'taxRegime': 601, 'rfc': 'BBA830831LJ2', 'nam...",...,None,[],None,[],[],2023-01-27 01:27:57,2023-01-28 06:11:48,"[{'identificationNumber': None, 'productIdenti...",True,True


In [159]:
DataFrame(df_items[1].dropna().to_list())


,identificationNumber,productIdentification,description,unitAmount,unitCode,quantity,discountAmount,taxType,taxRate,taxAmount,totalAmount,transferredTaxes,retainedTaxes,taxes
0,None,84121500,INTERESES NO SUJETOS A IVA,333.11,E48,1,0,None,None,None,333.11,"{'valueAddedTax': 0, 'sinTax': 0}","{'valueAddedTax': 0, 'incomeTax': 0, 'sinTax': 0}",[]
1,None,84121500,INTERESES NO SUJETOS A IVA,320.64,E48,1,0,None,None,None,320.64,"{'valueAddedTax': 0, 'sinTax': 0}","{'valueAddedTax': 0, 'incomeTax': 0, 'sinTax': 0}",[]
2,None,84121500,INTERESES NO SUJETOS A IVA,186.96,E48,1,0,None,None,None,186.96,"{'valueAddedTax': 0, 'sinTax': 0}","{'valueAddedTax': 0, 'incomeTax': 0, 'sinTax': 0}",[]


In [150]:
DataFrame(df_items["transferredTaxes"].to_list())


,valueAddedTax,sinTax
0,469.62,0
1,0.00,0


In [4]:
sat_credentials = get_metadata()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials/metadata.json'

In [26]:
get_all_credentials()

,@id,@type,id,type,rfc,status,metadata,createdAt,updatedAt
0,/credentials/9852142f-6a9e-4313-8eb9-20110fe33101,Credential,9852142f-6a9e-4313-8eb9-20110fe33101,ciec,MIGO010729SX9,valid,[],2023-01-27 01:28:54,2023-01-27 01:29:04
1,/credentials/98521378-055a-4a85-aa24-62a4a6ff6e44,Credential,98521378-055a-4a85-aa24-62a4a6ff6e44,ciec,SAMS990714DN1,valid,[],2023-01-27 01:26:54,2023-01-27 01:27:05
2,/credentials/9852134e-f056-4f6f-b081-9c8c36ee3a49,Credential,9852134e-f056-4f6f-b081-9c8c36ee3a49,ciec,MEAX800511V78,valid,[],2023-01-27 01:26:27,2023-01-27 01:26:36
3,/credentials/98520d85-26ea-4084-bf4f-bf87f3fe4f3e,Credential,98520d85-26ea-4084-bf4f-bf87f3fe4f3e,ciec,RUMC810130FP2,valid,[],2023-01-27 01:10:16,2023-01-27 01:10:25
4,/credentials/98520b90-c862-48b8-8f89-94ec32e5febb,Credential,98520b90-c862-48b8-8f89-94ec32e5febb,ciec,RIGC6106151I7,valid,[],2023-01-27 01:04:48,2023-01-27 01:04:58


In [5]:
{
    '@context': '/contexts/Entrypoint',
    '@id': '/',
    '@type': 'Entrypoint',
    'certificate': '/certificates',
    'credential': '/credentials',
    'event': '/events',
    'export': '/exports',
    'extraction': '/extractions',
    'invoice': '/invoices/issued',
    'invoiceBatchPayment': '/invoices/batch-payments',
    'invoiceCreditNote': '/invoices/credit-notes',
    'invoicePayment': '/invoices/payments',
    'link': '/links',
    'organization': '/organizations',
    'scheduler': '/schedulers',
    'schedulerRule': '/schedulers/rules',
    'taxRetention': '/tax-retentions',
    'taxStatus': '/tax-status',
    'taxpayer': '/taxpayers',
    'taxpayerGroup': '/taxpayer-groups',
    'user': '/users',
    'webServiceRequest': '/web-service-requests',
    'webhookEndpoint': '/webhook-endpoints',
    'webhookRequest': '/webhook-requests'
    }


{'@context': '/contexts/Entrypoint',
 '@id': '/',
 '@type': 'Entrypoint',
 'certificate': '/certificates',
 'credential': '/credentials',
 'event': '/events',
 'export': '/exports',
 'extraction': '/extractions',
 'invoice': '/invoices/issued',
 'invoiceBatchPayment': '/invoices/batch-payments',
 'invoiceCreditNote': '/invoices/credit-notes',
 'invoicePayment': '/invoices/payments',
 'link': '/links',
 'organization': '/organizations',
 'scheduler': '/schedulers',
 'schedulerRule': '/schedulers/rules',
 'taxRetention': '/tax-retentions',
 'taxStatus': '/tax-status',
 'taxpayer': '/taxpayers',
 'taxpayerGroup': '/taxpayer-groups',
 'user': '/users',
 'webServiceRequest': '/web-service-requests',
 'webhookEndpoint': '/webhook-endpoints',
 'webhookRequest': '/webhook-requests'}

In [54]:
http_response = get(**http_request)
response = loads(http_response.text)
print(response)

{'@context': '/contexts/Invoice', '@id': '/taxpayers/9852143e-a75e-4eff-8202-a0f615d76ea0/invoices', '@type': 'hydra:Collection', 'hydra:member': [{'@id': '/invoices/985a85b6-a497-45f3-84d1-34bd86720584', '@type': 'Invoice', 'id': '985a85b6-a497-45f3-84d1-34bd86720584', 'uuid': '42a87e7a-2500-42fd-afb3-fb89dd16603b', 'version': 3.3, 'type': 'I', 'usage': 'G03', 'paymentType': 'PUE', 'paymentMethod': '03', 'placeOfIssue': '64102', 'issuer': {'taxRegime': 612, 'rfc': 'MIGO010729SX9', 'name': 'OSVALDO MISSAEL MIRELES GUERRERO', 'blacklistStatus': None}, 'isIssuer': True, 'receiver': {'rfc': 'TCR0511087J0', 'name': 'TRANSPORTES CRUSAL SA DE CV', 'blacklistStatus': None}, 'isReceiver': False, 'currency': 'MXN', 'discount': 0, 'tax': 9453.45, 'retainedTaxes': {'total': 0, 'localTaxes': 0, 'valueAddedTax': 0, 'incomeTax': 0, 'sinTax': 0}, 'transferredTaxes': {'total': 9453.45, 'localTaxes': 0, 'valueAddedTax': 9453.45, 'sinTax': 0}, 'appliedTaxes': 9453.45, 'subtotal': 59084.11, 'total': 6853

In [57]:
response.keys()

dict_keys(['@context', '@id', '@type', 'hydra:member', 'hydra:totalItems', 'hydra:view', 'hydra:search'])

In [59]:
df = DataFrame(response["hydra:member"])


In [63]:
df['createdAt'] = to_datetime(df['createdAt'], format='%Y-%m-%d %H:%M:%S')
